### Read the dataframe by brute-force and check the reformating column names.

In [9]:
import polars as pl
import re
import numpy as np


def reformat_column_names(col_name: str) -> str:
    """A function for reformating the names of the columns. If some column' name is made up of words, seperate these words by _."""

    reformatted_name = re.sub(r"(?<![A-Z])(?<!^)([A-Z])", repl = r"_\1", string = col_name)



    return reformatted_name

laptop_data : pl.DataFrame = pl.read_csv(source = "laptopData.csv", # Read the data by brute force.
                                         ignore_errors = True)



data_cols: list[str] = laptop_data.columns

data_cols[data_cols.index("Unnamed: 0")] = "OrderID"

laptop_data.columns = np.vectorize(reformat_column_names)(data_cols)

data_cols: list[str] = laptop_data.columns

In [17]:
laptop_data

Order_ID,Company,Type_Name,Inches,Screen_Resolution,Cpu,Ram,Memory,Gpu,Op_Sys,Weight,Price
i64,str,str,f64,str,str,str,str,str,str,str,f64
0,"""Apple""","""Ultrabook""",13.3,"""IPS Panel Retina Display 2560x…","""Intel Core i5 2.3GHz""","""8GB""","""128GB SSD""","""Intel Iris Plus Graphics 640""","""macOS""","""1.37kg""",71378.6832
1,"""Apple""","""Ultrabook""",13.3,"""1440x900""","""Intel Core i5 1.8GHz""","""8GB""","""128GB Flash Storage""","""Intel HD Graphics 6000""","""macOS""","""1.34kg""",47895.5232
2,"""HP""","""Notebook""",15.6,"""Full HD 1920x1080""","""Intel Core i5 7200U 2.5GHz""","""8GB""","""256GB SSD""","""Intel HD Graphics 620""","""No OS""","""1.86kg""",30636.0
3,"""Apple""","""Ultrabook""",15.4,"""IPS Panel Retina Display 2880x…","""Intel Core i7 2.7GHz""","""16GB""","""512GB SSD""","""AMD Radeon Pro 455""","""macOS""","""1.83kg""",135195.336
4,"""Apple""","""Ultrabook""",13.3,"""IPS Panel Retina Display 2560x…","""Intel Core i5 3.1GHz""","""8GB""","""256GB SSD""","""Intel Iris Plus Graphics 650""","""macOS""","""1.37kg""",96095.808
…,…,…,…,…,…,…,…,…,…,…,…
1298,"""Lenovo""","""2 in 1 Convertible""",14.0,"""IPS Panel Full HD / Touchscree…","""Intel Core i7 6500U 2.5GHz""","""4GB""","""128GB SSD""","""Intel HD Graphics 520""","""Windows 10""","""1.8kg""",33992.64
1299,"""Lenovo""","""2 in 1 Convertible""",13.3,"""IPS Panel Quad HD+ / Touchscre…","""Intel Core i7 6500U 2.5GHz""","""16GB""","""512GB SSD""","""Intel HD Graphics 520""","""Windows 10""","""1.3kg""",79866.72
1300,"""Lenovo""","""Notebook""",14.0,"""1366x768""","""Intel Celeron Dual Core N3050 …","""2GB""","""64GB Flash Storage""","""Intel HD Graphics""","""Windows 10""","""1.5kg""",12201.12
